In [6]:
import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:
user = "entries_user"
password = "entries_password"
host = "localhost"
port = "5432"
database = "entries_db"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")

query_insert_intervals_tgt = """
    INSERT INTO intervals_tgt (id, full_name, enter_dt, exit_dt)
    WITH base AS (
        SELECT
            *,
            DATE(event_dt) AS event_dt_date
        FROM entries_src
        WHERE status IN ('Вход', 'Выход', 'Доступ запрещён')
    ),

    pairs AS (
        SELECT 
            full_name,
            event_dt,
            status,
            LEAD(status, 1) OVER (PARTITION BY full_name, DATE(event_dt) ORDER BY event_dt) AS next_status,
            LEAD(event_dt, 1) OVER (PARTITION BY full_name, DATE(event_dt) ORDER BY event_dt) AS next_dt,
            LEAD(status, 2) OVER (PARTITION BY full_name, DATE(event_dt) ORDER BY event_dt) AS next2_status,
            LEAD(event_dt, 2) OVER (PARTITION BY full_name, DATE(event_dt) ORDER BY event_dt) AS next2_dt,

            CASE 
                WHEN status IN ('Выход', 'Доступ запрещён') AND
                    LEAD(status) OVER (PARTITION BY full_name, event_dt_date ORDER BY event_dt) = status AND
                    EXTRACT(EPOCH FROM (LEAD(event_dt) OVER (PARTITION BY full_name, event_dt_date ORDER BY event_dt) - event_dt)) <= 60
                THEN true
            
                ELSE false
            END AS is_exit_duplicate
        FROM base
    ),

    intervals AS (
        SELECT 
            full_name,

            CASE 
                WHEN status = 'Вход' AND next_status = 'Вход'
                    AND EXTRACT(EPOCH FROM (next_dt - event_dt)) <= 60
                THEN next_dt

                WHEN status = 'Вход' AND next_status IN ('Выход', 'Доступ запрещён')
                THEN event_dt

                ELSE NULL
            END AS enter_dt,

            CASE
                WHEN status = 'Вход' AND next_status = 'Вход'
                    AND EXTRACT(EPOCH FROM (next_dt - event_dt)) <= 60
                    AND next2_status IN ('Выход', 'Доступ запрещён')
                THEN next2_dt

                WHEN status = 'Вход' AND next_status IN ('Выход', 'Доступ запрещён')
                THEN next_dt

                ELSE NULL
            END AS exit_dt
        FROM pairs
    )

    SELECT DISTINCT ON (full_name, enter_dt)
        ROW_NUMBER() OVER (ORDER BY full_name, enter_dt) AS id,
        full_name,
        enter_dt,
        exit_dt
    FROM intervals
    WHERE enter_dt IS NOT NULL
    ORDER BY full_name, enter_dt;
"""

with engine.connect() as conn:
    with conn.begin():
        res = conn.execute(text(query_insert_intervals_tgt))


In [57]:
query_insert_workdays_tgt = """
    WITH params AS (
        SELECT
            full_name,
            DATE(enter_dt) AS report_date,
            MIN(enter_dt) AS first_enter_dt
        FROM intervals_tgt
        WHERE DATE(enter_dt) =  DATE '2006-04-13'
        GROUP BY full_name, DATE(enter_dt)
    ),
    last_enter AS (
        SELECT
            t.full_name,
            DATE(t.enter_dt) AS report_date,
            t.enter_dt AS last_enter_dt,
            t.exit_dt
        FROM intervals_tgt t
        JOIN params p ON t.full_name = p.full_name
            AND DATE(t.enter_dt) = p.report_date
        WHERE t.enter_dt <= (p.report_date::timestamp + INTERVAL '1 day 4 hours')
    ),
    last_enter_max AS (
        SELECT
            full_name,
            report_date,
            MAX(last_enter_dt) AS last_enter_dt
        FROM last_enter
        GROUP BY full_name, report_date
    )
    SELECT
        p.full_name,
        p.report_date,
        p.first_enter_dt,
        le.exit_dt
    FROM params p
    JOIN last_enter_max lm ON p.full_name = lm.full_name AND p.report_date = lm.report_date
    JOIN last_enter le ON le.full_name = lm.full_name AND le.report_date = lm.report_date AND le.last_enter_dt = lm.last_enter_dt
    ORDER BY p.full_name, p.report_date;
"""

with engine.connect() as conn:
    with conn.begin():
        res = conn.execute(text(query_insert_workdays_tgt))

cnt = 0
for row in res:
    print(row)
    cnt += 1

print(cnt)

('Головин Сергей Вальерьевич', datetime.date(2006, 4, 13), datetime.datetime(2006, 4, 13, 10, 26, 11), datetime.datetime(2006, 4, 13, 12, 46, 52))
('Дружинин Георгий Михайлович', datetime.date(2006, 4, 13), datetime.datetime(2006, 4, 13, 9, 22, 10), datetime.datetime(2006, 4, 13, 18, 28, 43))
('Лапухов Алексей Дмитриевич', datetime.date(2006, 4, 13), datetime.datetime(2006, 4, 13, 8, 10, 55), datetime.datetime(2006, 4, 13, 19, 17, 51))
('Путилов Андрей Маркович', datetime.date(2006, 4, 13), datetime.datetime(2006, 4, 13, 10, 26, 18), datetime.datetime(2006, 4, 13, 18, 22, 50))
('Семёнов Дмитрий Иванович', datetime.date(2006, 4, 13), datetime.datetime(2006, 4, 13, 9, 10, 2), datetime.datetime(2006, 4, 13, 17, 27, 44))
5
